In [59]:
from IPython.display import HTML

from arcgis.gis import GIS
import arcgis.network as network

import urllib.parse as encode
import requests
import json

In [60]:
start = '96.217989,16.807021'
end = '96.207368,16.8128586'
stops = start+';'+end
encoded = encode.quote(stops)

In [1]:
import requests

url = "https://www.arcgis.com/sharing/rest/oauth2/token"

payload = "client_id=*************&client_secret=*****************************&grant_type=client_credentials"
headers = {
    'Content-Type': "application/x-www-form-urlencoded"
    }

response = requests.request(url,data=headers)

print(response.text)

TypeError: request() missing 1 required positional argument: 'url'

In [81]:
def get_route(encoded_route):
    import datetime

    start_time = int(datetime.datetime.utcnow().timestamp() * 1000)
    
    url = "https://route.arcgis.com/arcgis/rest/services/World/Route/NAServer/Route_World/solve"

    payload = "f=json&token=*********************************************&stops="+encoded_route
    headers = {
    'Content-Type': "application/x-www-form-urlencoded"
        'directions_language':'en-US',
        'start_time':str(start_time),
        'start_time_is_utc':'True'
        }

    response = requests.request("POST", url, data=payload, headers=headers)

    return(response.text)

In [82]:
data = json.loads(get_route(encoded_route=encoded))

In [83]:
type(data)

dict

In [85]:
import datetime
records = []
travel_time, time_counter = 0, 0
distance, distance_counter = 0, 0

for i in data['directions'][0]['features']:
    time_of_day = datetime.datetime.fromtimestamp(i['attributes']['arriveTimeUTC'] / -1000).strftime('%H:%M:%S')
    time_counter = i['attributes']['time']
    distance_counter = i['attributes']['length']
    travel_time += time_counter
    distance += distance_counter
    records.append( (time_of_day, i['attributes']['text'], 
                     round(travel_time, 2), round(distance, 2))  )    

In [67]:
pd.set_option('display.max_colwidth', 100)
df = pd.DataFrame.from_records(records, index=[i for i in range(1, len(records) + 1)], 
                               columns=['Time of day', 'Direction text', 
                                        'Duration (min)', 'Distance (km)'])
HTML(df.to_html(index=False))

Time of day,Direction text,Duration (min),Distance (km)
06:30:00,Start at Location 1,0.00,0.00
06:30:00,Go west,0.05,0.02
06:30:00,Keep left at the fork,1.83,0.85
06:30:00,Turn left,2.56,1.10
06:30:00,Turn right,3.01,1.24
06:30:00,Turn left,4.05,1.59
06:30:00,Turn left,4.38,1.70
06:30:00,"Finish at Location 2, on the left",4.38,1.70


In [58]:
data['directions'][0]['features'][0]['attributes']['arriveTimeUTC'] / 1000

-2209161600.0

In [87]:
import datetime
records = []
travel_time, time_counter = 0, 0
distance, distance_counter = 0, 0
for i in data['directions'][0]['features']:
    time_counter = i['attributes']['time']
    distance_counter = i['attributes']['length']
    travel_time += time_counter
    distance += distance_counter
    records.append( (i['attributes']['text'], 
                     round(travel_time, 2), round(distance, 2))  )  
    

In [89]:
records

[('Start at Location 1', 0, 0),
 ('Go west', 0.05, 0.02),
 ('Keep left at the fork', 1.83, 0.85),
 ('Turn left', 2.56, 1.1),
 ('Turn right', 3.01, 1.24),
 ('Turn left', 4.05, 1.59),
 ('Turn left', 4.38, 1.7),
 ('Finish at Location 2, on the left', 4.38, 1.7)]

In [91]:
pd.set_option('display.max_colwidth', 100)
df = pd.DataFrame.from_records(records, index=[i for i in range(1, len(records) + 1)], 
                               columns=['Direction text', 
                                        'Duration (min)', 'Distance (km)'])
HTML(df.to_html(index=False))

Direction text,Duration (min),Distance (km)
Start at Location 1,0.00,0.00
Go west,0.05,0.02
Keep left at the fork,1.83,0.85
Turn left,2.56,1.10
Turn right,3.01,1.24
Turn left,4.05,1.59
Turn left,4.38,1.70
"Finish at Location 2, on the left",4.38,1.70


In [112]:
data["directions"][0]["summary"]


{'totalLength': 1.6987171545061681,
 'totalTime': 4.383021060815048,
 'totalDriveTime': 4.383021060815048,
 'envelope': {'xmin': 96.20610000000005,
  'ymin': 16.806700000000035,
  'xmax': 96.21839000000006,
  'ymax': 16.817310000000077,
  'spatialReference': {'wkid': 4326, 'latestWkid': 4326}}}

In [133]:
for i in data['routes']['features'][0]['geometry']['paths'][0]:
    print(i)

[96.21787331400009, 16.807301953000035]
[96.21782000000007, 16.80728000000005]
[96.21766000000008, 16.80705000000006]
[96.21781000000004, 16.806730000000073]
[96.21820000000008, 16.806700000000035]
[96.21839000000006, 16.807100000000048]
[96.21816000000007, 16.807330000000036]
[96.21793000000008, 16.80768000000006]
[96.21787000000006, 16.80812000000003]
[96.21784000000008, 16.808290000000056]
[96.21777000000009, 16.80878000000007]
[96.21771000000007, 16.80923000000007]
[96.21764000000007, 16.809700000000078]
[96.21757000000008, 16.810110000000066]
[96.21748000000008, 16.810580000000073]
[96.21738000000005, 16.811040000000048]
[96.21726000000007, 16.811470000000043]
[96.21707000000004, 16.81191000000007]
[96.21693000000005, 16.812280000000044]
[96.21507000000008, 16.817130000000077]
[96.21500000000003, 16.817310000000077]
[96.21369000000004, 16.81669000000005]
[96.21325000000007, 16.816380000000038]
[96.21281000000005, 16.816050000000075]
[96.21235000000007, 16.815710000000024]
[96.2119

In [134]:
import numpy as np

In [136]:
np.array(data['routes']['features'][0]['geometry']['paths'][0])

array([[96.21787331, 16.80730195],
       [96.21782   , 16.80728   ],
       [96.21766   , 16.80705   ],
       [96.21781   , 16.80673   ],
       [96.2182    , 16.8067    ],
       [96.21839   , 16.8071    ],
       [96.21816   , 16.80733   ],
       [96.21793   , 16.80768   ],
       [96.21787   , 16.80812   ],
       [96.21784   , 16.80829   ],
       [96.21777   , 16.80878   ],
       [96.21771   , 16.80923   ],
       [96.21764   , 16.8097    ],
       [96.21757   , 16.81011   ],
       [96.21748   , 16.81058   ],
       [96.21738   , 16.81104   ],
       [96.21726   , 16.81147   ],
       [96.21707   , 16.81191   ],
       [96.21693   , 16.81228   ],
       [96.21507   , 16.81713   ],
       [96.215     , 16.81731   ],
       [96.21369   , 16.81669   ],
       [96.21325   , 16.81638   ],
       [96.21281   , 16.81605   ],
       [96.21235   , 16.81571   ],
       [96.21193   , 16.81539   ],
       [96.21163   , 16.81575   ],
       [96.21135   , 16.81609   ],
       [96.211     ,